In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

In [3]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

1. read data

In [4]:
df=pd.read_csv("data/StudentsPerformance.csv")

In [5]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


2.take i/p x and output to be predicted be y
    take  y to be one of the feature and remaining feature to be x

In [7]:
x=df.drop(columns=["math_score"])
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [19]:
y=df["math_score"]
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

take numerical feature and categorical feature into two class to convert into two categories to be studied

In [9]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   reading_score                1000 non-null   int64 
 6   writing_score                1000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [10]:
cat_feat=x.select_dtypes(include=["object"]).columns
cat_feat

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [11]:
num_feat=x.select_dtypes(exclude=["object"]).columns
num_feat

Index(['reading_score', 'writing_score'], dtype='object')

In [12]:
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

In [13]:
numeric_transformer=StandardScaler()
categoric_transformer=OneHotEncoder()

In [14]:
preprocessor=ColumnTransformer(
    [
        ("ohe",categoric_transformer,cat_feat),
        ("ssclr",numeric_transformer,num_feat)
    ]
)

In [15]:
X=preprocessor.fit_transform(x)

In [16]:
X.shape

(1000, 19)

In [20]:
#seperate dataset into train and test
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [21]:
X_train.shape,X_test.shape

((800, 19), (200, 19))

*evaluate models and functions

In [22]:
def evaluate_model(actual,pred):
    mae=mean_absolute_error(actual,pred)
    mse=mean_squared_error(actual,pred)
    rmse=np.sqrt(mse)
    r2_sco=r2_score(actual,pred)
    return mae, rmse, r2_sco

In [23]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso":Lasso(),
    "Ridge": Ridge(),
    "k-neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegrssor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list=[]
r2_list=[]

In [24]:
list(models.keys())
list(models.values())

[LinearRegression(),
 Lasso(),
 Ridge(),
 KNeighborsRegressor(),
 DecisionTreeRegressor(),
 RandomForestRegressor(),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
 AdaBoostRegressor()]

In [25]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    #print(model)
    model.fit(X_train,y_train)    #train model
    # prediction
    y_train_prediction=model.predict(X_train)
    y_test_prediction=model.predict(X_test)
    
    #evaluate train and test datase
    # for training
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_prediction)
    model_test_mae, model_test_rmse, model_test_r2=evaluate_model(y_test,y_test_prediction)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model Performance for training set")
    print("-Root Mean Squared Error: {:4f}".format(model_train_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("-R2-score : {:.4f}".format(model_train_r2))

    print("------------------------------------------")

    print("Model Performance for test set")
    print("-Root Mean Squared Error: {:4f}".format(model_test_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("-R2-score : {:.4f}".format(model_test_r2))

    


Linear Regression
Model Performance for training set
-Root Mean Squared Error: 5.327633
-Mean Absolute Error: 4.2691
-R2-score : 0.8741
------------------------------------------
Model Performance for test set
-Root Mean Squared Error: 5.418484
-Mean Absolute Error: 4.2234
-R2-score : 0.8793
Lasso
Model Performance for training set
-Root Mean Squared Error: 6.593816
-Mean Absolute Error: 5.2063
-R2-score : 0.8071
------------------------------------------
Model Performance for test set
-Root Mean Squared Error: 6.519695
-Mean Absolute Error: 5.1579
-R2-score : 0.8253
Ridge
Model Performance for training set
-Root Mean Squared Error: 5.323325
-Mean Absolute Error: 4.2650
-R2-score : 0.8743
------------------------------------------
Model Performance for test set
-Root Mean Squared Error: 5.390387
-Mean Absolute Error: 4.2111
-R2-score : 0.8806
k-neighbors Regressor
Model Performance for training set
-Root Mean Squared Error: 5.709063
-Mean Absolute Error: 4.5175
-R2-score : 0.8554
-----

In [26]:
print(model_list)

['Linear Regression', 'Lasso', 'Ridge', 'k-neighbors Regressor', 'Decision Tree', 'Random Forest Regressor', 'XGBRegrssor', 'CatBoost Regressor', 'AdaBoost Regressor']


In [27]:
#solve the error in the below line
#pd.DataFrame(list(zip(model_list,r2_list),columns=["Model Name","r2_score"]).sort_values(by=["r2_score"],ascending=False))